Bài tập thực hành 3: BÀI TOÁN IMAGE RETRIEVAL

Họ tên: Nguyễn Nhật Trường

MSSV: 20522087

Ngày hoàn thành: 05/01/2022

In [1]:
# Import nesscesary libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow

In [2]:
%cd "/content/drive/MyDrive/Colab Notebooks/cs331_advanced_cv"
database_path = "./CIR/database"
query_path = "./CIR/query"

/content/drive/MyDrive/Colab Notebooks/cs331_advanced_cv


# YÊU CẦU 1

---


*   Sử dụng đặc trưng Histogram để mô tả các ảnh.
*   Sử dụng phương pháp tính khoảng cách cosin giữa ảnh truy vấn và tập ảnh trong database
*   Tính Precision@10 và Recall@10 cho 15 ảnh truy vấn.
*   Minh họa 10 kết quả khi thực hiện truy vấn bằng ảnh “ant_02.jpg”







## SỬ DỤNG ĐẶC TRƯNG HISTTOGRAM ĐỂ MÔ TẢ CÁC ẢNH

In [3]:
# SỬ DỤNG ĐẶC TRƯNG HISTTOGRAM ĐỂ MÔ TẢ CÁC ẢNH
BINS = 264
def image_hist(path):
    his = []
    for img_name in tqdm(os.listdir(path)):
        img = cv2.imread(os.path.join(path, img_name), cv2.COLOR_BGR2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_his, bins = np.histogram(img, bins=BINS)
        his.append(img_his)
    return np.array(his)

In [4]:
database_his = image_hist(database_path)
query_his = image_hist(query_path)

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

## SỬ DỤNG PHƯƠNG PHÁP TÍNH KHOẢNG CÁCH COSIN GIỮA ẢNH TRUY VẤN VÀ TẬP ẢNH TRONG DATABASE


In [5]:
cosine_distances = cosine_similarity(query_his, database_his)
print("Cosine Distance: ", cosine_distances)
print("Cosine Distance shape: ", cosine_distances.shape)

Cosine Distance:  [[0.93625783 0.10277407 0.14756929 ... 0.24899055 0.25515579 0.35311698]
 [0.98146475 0.08582446 0.13762139 ... 0.25054034 0.21161848 0.37139864]
 [0.98228169 0.08814273 0.15364318 ... 0.2406278  0.20279488 0.33479719]
 ...
 [0.10254908 0.94448006 0.84680713 ... 0.00217212 0.00593794 0.00223205]
 [0.1019179  0.94324509 0.84380733 ... 0.00133469 0.00466457 0.00159007]
 [0.10235314 0.9432834  0.8443012  ... 0.00151924 0.00499319 0.00164497]]
Cosine Distance shape:  (18, 60)


## TÍNH PRECISION@10 VÀ RECALL@10 CHO 15 ẢNH TRUY VẤN.


In [6]:
result_dict = dict()
for i, query_distances in enumerate(cosine_distances):
    # -------------------------------------------------
    query_label = os.listdir(query_path)[i].split("_")[0]
    # -------------------------------------------------
    gt_label, pred_label = [], []

    for database_idx, distance in enumerate(query_distances):
        # -------------------------------------------------       
        database_label = os.listdir(database_path)[database_idx].split("_")[0]        
        # -------------------------------------------------
        gt_label.append(1 \
                          if database_label == query_label \
                          else 0)        
        # -------------------------------------------------
        pred_label.append(1 \
                               if distance >= 0.1 \
                               else 0)
    # -------------------------------------------------
    result_dict[os.listdir(query_path)[i]] = [
        precision_score(gt_label, pred_label), 
        recall_score(gt_label, pred_label)
    ]

In [7]:
# Show result
result = pd.DataFrame.from_dict(result_dict, orient='index', columns=['Precison@10', 'Recall@10'])
result

,Precison@10,Recall@10
accordion_01.jpg,0.175439,1.0
accordion_03.jpg,0.188679,1.0
accordion_02.jpg,0.188679,1.0
airplane_02.jpg,0.188679,1.0
airplane_01.jpg,0.204082,1.0
airplane_03.jpg,0.208333,1.0
anchor_02.jpg,0.137931,0.8
ant_03.jpg,0.175439,1.0
anchor_01.jpg,0.107143,0.6
anchor_03.jpg,0.125000,0.7


## MINH HỌA 10 KẾT QUẢ KHI THỰC HIỆN TRUY VẤN BẰNG ẢNH “ANT_02.JPG”


In [8]:
img_name = "ant_02.jpg"
img = cv2.cvtColor(cv2.imread(os.path.join(query_path, img_name)), cv2.COLOR_BGR2RGB)
print("Query image: ")
cv2_imshow(img)

index = os.listdir(query_path).index(img_name)
distances = np.copy(cosine_distances[index]) 
ind = np.argpartition(distances, -10)[-10:]

print("\n\n","Result of query: ")
for pos, i in enumerate(ind): 
    img_name = os.listdir(database_path)[i]
    img = cv2.imread(os.path.join(database_path, img_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2_imshow(img)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# YÊU CẦU 2

---
Minh họa 10 kết quả khi thực hiện truy vấn bằng ảnh “ant_02.jpg”.


## Minh họa 10 kết quả khi thực hiện truy vấn bằng ảnh “ant_02.jpg”.

In [9]:
img_name = "ant_02.jpg"
img = cv2.cvtColor(cv2.imread(os.path.join(query_path, img_name)), cv2.COLOR_BGR2RGB)
print("Query image: ")
cv2_imshow(img)

index = os.listdir(query_path).index(img_name)
distances = np.copy(cosine_distances[index]) 
ind = np.argpartition(distances, -10)[-10:]

print("\n\n","Result of query: ")
for pos, i in enumerate(ind): 
    img_name = os.listdir(database_path)[i]
    img = cv2.imread(os.path.join(database_path, img_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    cv2_imshow(img)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

# YÊU CẦU 3

---




*   Tính độ đo MAP@10
*   Tính độ đo MRR cho 15 ảnh query




## Tính độ đo MAP@10

In [13]:
precisions = []
recalls = []

for query_idx, query_distances in enumerate(cosine_distances):
    query_label = os.listdir(query_path)[query_idx].split("_")[0]
    true_label = []
    predicted_label = []
    for database_idx, distance in enumerate(query_distances):
        database_label = os.listdir(database_path)[database_idx].split("_")[0]
        true_label.append(1 \
                          if database_label == query_label \
                          else 0)
        predicted_label.append(1 \
                               if distance >= 0.1 \
                               else 0)
    precisions.append(np.array([precision_score(true_label, predicted_label), 1]))
    recalls.append(np.array([recall_score(true_label, predicted_label), 0]))

precisions = np.array(precisions)
recalls = np.array(recalls)

APs = []
for i in range(len(precisions)):
    APs.append(np.sum((recalls[i][:-1] - recalls[i][1:]) * precisions[i][:-1]))
mAP = np.mean(APs)
print("Độ đo mAP@10: ", mAP)

Độ đo mAP@10:  0.12211285365337542


## Tính độ đo MRR cho 15 ảnh query

In [12]:
sum_rank = 0
for query_idx, query_distances in enumerate(cosine_distances):
    query_label = os.listdir(query_path)[query_idx].split("_")[0]
    des_arg = np.copy(query_distances)
    des_arg = np.argsort(des_arg)

    for i, arg in enumerate(des_arg[::-1]):
        db_label = os.listdir(database_path)[arg].split("_")[0]
        if (query_label == db_label):
            sum_rank += (1 / (i + 1))
            break

print("Độ đo MRR cho 15 ảnh query: ", sum_rank / cosine_distances.shape[0])

Độ đo MRR cho 15 ảnh query:  0.34338624338624346
